In [1]:
import cv2
import numpy as np
import os
from jupyter_clickable_image_widget import ClickableImageWidget
from IPython.display import display
from Functions.USBCamera import USBCamera
import ipywidgets as widgets
from datetime import datetime
from threading import Thread

class RealTimeHSVAnalyzer:
    def __init__(self, camera_width=640, camera_height=480):
        # Start kamera
        self.camera = USBCamera.instance(width=camera_width, height=camera_height)

        # Set up ClickableImageWidget for display and Label for HSV info
        self.image_widget = ClickableImageWidget(width=camera_width, height=camera_height)
        self.hsv_label = widgets.Label(value="Klikk på bildet for å se HSV-verdi")

        # Set up button for taking snapshots
        self.snapshot_button = widgets.Button(description="Ta bilde")
        self.snapshot_button.on_click(self.take_snapshot)

        # Display widgets
        display(self.image_widget)
        display(self.hsv_label)
        display(self.snapshot_button)

        # Start en tråd for kontinuerlig oppdatering av widgeten
        self.running = True
        self.update_thread = Thread(target=self.update_widget, daemon=True)
        self.update_thread.start()

        # Set up event handler for clicks
        self.image_widget.on_msg(self.on_click)
        
        # Prepare directory for saving snapshots
        self.save_directory = os.path.join(os.getcwd(), "snapshots")
        os.makedirs(self.save_directory, exist_ok=True)
        
        print("Kamera-feed med HSV-analyse på klikk er startet.")

    def convert_to_image_widget(self, frame):
        """Konverter bilde til en format egnet for visning i widgeten."""
        _, encoded_image = cv2.imencode('.jpg', frame)
        return encoded_image.tobytes()

    def update_widget(self):
        """Kontinuerlig oppdatering av widgeten med kamerarammen."""
        while self.running:
            frame = self.camera.value
            if frame is not None:
                self.frame = frame
                self.hsv_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2HSV)
                self.image_widget.value = self.convert_to_image_widget(frame)

    def on_click(self, widget, content, msg=None):
        """Håndter klikk for å vise HSV-verdi."""
        click_x, click_y = int(content['eventData']['offsetX']), int(content['eventData']['offsetY'])

        # Skaler koordinatene basert på widgetens dimensjoner
        x = int(click_x * (self.hsv_frame.shape[1] / self.image_widget.width))
        y = int(click_y * (self.hsv_frame.shape[0] / self.image_widget.height))

        # Sjekk om klikket er innenfor bildet
        if 0 <= x < self.hsv_frame.shape[1] and 0 <= y < self.hsv_frame.shape[0]:
            hsv_value = self.hsv_frame[y, x]
            self.hsv_label.value = f"HSV-verdi ved ({x}, {y}): H={hsv_value[0]}, S={hsv_value[1]}, V={hsv_value[2]}"
            print(self.hsv_label.value)
        else:
            print(f"Klikk utenfor bildet: ({x}, {y})")

    def take_snapshot(self, button):
        """Ta et snapshot og lagre det som en fil."""
        if self.frame is not None:
            filename = f"snapshot_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
            filepath = os.path.join(self.save_directory, filename)
            
            cv2.imwrite(filepath, cv2.cvtColor(self.frame, cv2.COLOR_RGB2BGR))
            print(f"Bilde lagret som {filepath}")
        else:
            print("Ingen ramme tilgjengelig å ta bilde av.")

    def stop(self):
        """Stopp kameraet og tråden."""
        self.running = False
        self.update_thread.join()
        self.camera.stop()
        print("Kameraet er stoppet.")

# Start HSV-analyse
analyzer = RealTimeHSVAnalyzer(camera_width=640, camera_height=480)


ClickableImageWidget(value=b'', height='480', width='640')

Label(value='Klikk på bildet for å se HSV-verdi')

Button(description='Ta bilde', style=ButtonStyle())

Kamera-feed med HSV-analyse på klikk er startet.
